In [223]:
import os, os.path
import pandas as pd
import re


# We are going to explore SQL Alchemy and Python

- SQL Alchemy allows you to interact directly with local or remote SQL database services using an engine
- Remote services often require setting up environment variables



###  New package: `pathlib`
- Create POSIX paths as objects
- Slightly cleaner than the `os.path` library for path management
- If/when you encounter urls, you can similarly use `urllib`

In [224]:
import pathlib

In [225]:
# create a path in using the absolute path to present directory
path_cur = pathlib.Path(os.path.abspath("."))


In [43]:
# join with elements using the .joinpath method
# note: enter subdirectories sequentially as arguments
path_tables = path_cur.joinpath("tables")


In [231]:
# check that it is a directoy
path_tables2.is_dir()


False

In [227]:
# check that it is a file
path_tables.is_file()


False

##  Let's explore SQLAlchemy a bit

- Imported using `sqlalchemy`
- We can read and interact with an existing database easily, including:
    - Local SQLite databases
    - Remote services like Snowflake, Google, Microsoft, and Amazon
        - These services often require a username, access key (region, secret access key, secret key, etc.) to connect
        - These can be set in the environment or in configuration files
        - One note: **NEVER** save your access keys directly to a jupyter notebook or file in a repository. Place them elsewhere, somewhere inaccessible (or less accessible at the least) by remote actors
- Learn more [about SQLAlchemy](https://www.sqlalchemy.org/)
  

In [232]:
import sqlalchemy

In [233]:
# path to the database
path_demo = path_cur.joinpath("intermediate_database.sqlite")



In [45]:
# TODO: verify that the path exists


###  SQL Alchemy uses the `Engine` object to connect to a database

- Engines are created using the `.create_engine()` function
- The engine can create connections to the database and execute queries
- For SQLite, we create an engine using the `sqlite:///` notation as an argument to `sqlalchemy.create_engine`
    - E.g., `fp = "PATH/TO/DB.sqlite"`, `sqlalchemy.create_engine(f"sqlite:///{fp}")` 

In [234]:
engine = sqlalchemy.create_engine(f"sqlite:///{path_demo}")

In [235]:
# use an Inspector object to check the engine
inspector = sqlalchemy.inspect(engine)

In [236]:
# let's see the table names
inspector.get_table_names()


['AccumulatedAnnualDemand',
 'AnnualEmissionLimit',
 'AnnualExogenousEmission',
 'AvailabilityFactor',
 'CapacityOfOneTechnologyUnit',
 'CapacityToActivityUnit',
 'CapitalCost',
 'CapitalCostStorage',
 'DefaultParams',
 'DepreciationMethod',
 'DiscountRate',
 'EMISSION',
 'EmissionActivityRatio',
 'EmissionsPenalty',
 'FUEL',
 'FixedCost',
 'InputActivityRatio',
 'InterestRateStorage',
 'InterestRateTechnology',
 'LTsGroup',
 'MODE_OF_OPERATION',
 'MinShareProduction',
 'MinStorageCharge',
 'MinimumUtilization',
 'ModelPeriodEmissionLimit',
 'ModelPeriodExogenousEmission',
 'NODE',
 'NodalDistributionDemand',
 'NodalDistributionStorageCapacity',
 'NodalDistributionTechnologyCapacity',
 'OperationalLife',
 'OperationalLifeStorage',
 'OutputActivityRatio',
 'REGION',
 'REGIONGROUP',
 'REMinProductionTarget',
 'REMinProductionTargetRG',
 'RETagTechnology',
 'RRGroup',
 'RampRate',
 'RampingReset',
 'ReserveMargin',
 'ReserveMarginTagFuel',
 'ReserveMarginTagTechnology',
 'ResidualCapacity

In [237]:
# let's try running a query. Pull the EMISSION table
query = "SELECT * FROM EMISSION;"

In [238]:
# open a connection - use with block to avoid issues with closing connection when done with transactions
# try executing the query. What happens?
with engine.connect() as con:
    result = con.execute(query)

ObjectNotExecutableError: Not an executable object: 'SELECT * FROM EMISSION;'

In [239]:
# SQLAlchemy requires you to convert the query to an sqlalchemy.text type
query_text = sqlalchemy.text(query)
?query_text

Type:        TextClause
String form: SELECT * FROM EMISSION;
File:        /opt/miniconda3/envs/amber_is_your_energy/lib/python3.11/site-packages/sqlalchemy/sql/elements.py
Docstring:  
Represent a literal SQL text fragment.

E.g.::

    from sqlalchemy import text

    t = text("SELECT * FROM users")
    result = connection.execute(t)

The :class:`_expression.TextClause` construct is produced using the
:func:`_expression.text`
function; see that function for full documentation.

.. seealso::

    :func:`_expression.text`

In [240]:
# we can execute a simple query here
with engine.connect() as con:
    result = con.execute(sqlalchemy.text(query))

In [241]:
result

In [242]:
# the result of a "SELECT" can be used to iterate through returns. What are they?
for row in result:
    print(row)

('c2f6', 'Perfluoroethane (PFC-116)')
('c2h3f3', '1,1,1-Trifluoroethane (HFC-143a)')
('c2hf5', 'Pentafluoroethane (HFC-125)')
('c3f8', 'Perfluoropropane (PFC-218)')
('c3h2f6', '1,1,1,3,3,3-Hexafluoropropane (HFC-236fa)')
('c3h3f5', 'Pentafluoropropane (HFC-245fa)')
('c3hf7', '1,1,1,2,3,3,3-Heptafluoropropane (HFC-227ea)')
('c4f10', 'Perflubutane/Decafluorobutane (PFC-31-10)')
('c4f6', 'Hexafluorobutadiene (PFC-1114)')
('c4f8o', 'Octafluorooxolane/Octafluorotetrahydrofuran')
('c4h5f5', '1,1,1,3,3-Pentafluorobutane (HFC-365mfc)')
('c5f12', 'Dodecafluoropentane')
('c5f8', 'Octafluorocyclopentene (PFC-C-1418)')
('c5h2f10', '2,3-Dihydrodeca-fluoropentane (HFC-43-10mee)')
('c6f14', 'Perfluorohexane/Tetradecafluorohexane (PFC-51-14)')
('c7f16', 'Hexadecafluoroheptane')
('c8f18', 'Octadecafluorooctane')
('cc4f8', 'Perfluorocyclobutane (PFC-C-318)')
('cf4', 'Perfluoromethane (PFC-14)')
('ch2f2', 'Difluoromethane (HFC-32)')
('ch2fch2f', '1,2-Difluoroethane (HFC-152)')
('ch2fchf2', '1,1,2-Trifluo

###  Once we see a table, we can get information about the schema

Different SQL DBMS have slightly different approaches--for SQLite, we can use:
- `PRAGMA table_info(TABLE_NAME);`
- `SELECT sql FROM sqlite_schema WHERE name='TABLE_NAME';`

**Try both of these queries. What do you notice about the difference?**

In [251]:
TABLE_NAME = "EMISSION"
with engine.connect() as con:
    schema_result = con.execute(
        #sqlalchemy.text(f"PRAGMA table_info({TABLE_NAME});")
        sqlalchemy.text(f"SELECT sql FROM sqlite_schema WHERE name='{TABLE_NAME}'")
    )

In [252]:
for row in schema_result:
    print(row)

('CREATE TABLE `EMISSION` ( `val` TEXT NOT NULL UNIQUE, `desc` TEXT, PRIMARY KEY(`val`) )',)


###  For retrievals (`SELECT`), we can use built in functionality from Pandas to call tables as DataFrames
- Use `pd.read_sql_query(query, connection)`

In [253]:
with engine.connect() as con:
    df_out = pd.read_sql_query(query, con)
    
df_out

,val,desc
0,c2f6,Perfluoroethane (PFC-116)
1,c2h3f3,"1,1,1-Trifluoroethane (HFC-143a)"
2,c2hf5,Pentafluoroethane (HFC-125)
3,c3f8,Perfluoropropane (PFC-218)
4,c3h2f6,"1,1,1,3,3,3-Hexafluoropropane (HFC-236fa)"
5,c3h3f5,Pentafluoropropane (HFC-245fa)
6,c3hf7,"1,1,1,2,3,3,3-Heptafluoropropane (HFC-227ea)"
7,c4f10,Perflubutane/Decafluorobutane (PFC-31-10)
8,c4f6,Hexafluorobutadiene (PFC-1114)
9,c4f8o,Octafluorooxolane/Octafluorotetrahydrofuran


# Now, let's add a table to the database

In [254]:
# read in one of the tables from 
table_name = "ATTRIBUTE_PRIMARY"
df_to_write = pd.read_csv(path_tables.joinpath(f"{table_name}.csv"))

In [256]:
table_name in sqlalchemy.inspect(engine).get_table_names()

False

In [257]:
# we can use the pd.to_sql function to write to the database
df_to_write.to_sql(
    table_name, 
    engine, 
    if_exists = "replace", 
    index = None,
);


In [258]:
# did it succeed?
table_name in sqlalchemy.inspect(engine).get_table_names()

True

In [259]:
with engine.connect() as con:
    df_primary = pd.read_sql_query(f"SELECT * FROM {table_name}''", con)
    
df_primary

,primary_id,design_id,strategy_id,future_id
0,0,0,0,0
1,40,0,0,40
2,14014,0,1014,0
3,14054,0,1014,40
4,118118,0,5001,0
5,118158,0,5001,40
6,121121,0,5004,0
7,121161,0,5004,40
8,123123,0,5006,0
9,123163,0,5006,40


In [260]:
# let's say we want to drop the table; write the query
query_drop = f"DROP TABLE {table_name};"


In [261]:
with engine.connect() as con:
    con.execute(sqlalchemy.text(query_drop))
    con.commit() # commit the changes explicitly



In [262]:
# is the table still there?
table_name in sqlalchemy.inspect(engine).get_table_names() 

False

# Explicit schema construction

We can--and should--explicitly construct schema for our databases when we know the structure. 

Check [SQLite datatypes](https://www.sqlite.org/datatype3.html) for more information on datatypes in SQLite.

In [263]:
# we can also be explicit about the schema we provide when we use pd.DataFrame.to_sql()
# suppose this dictionary maps pandas types to sql data types

dict_dtypes_to_sql_types = {
    "string": "STRING", # this could be character or varchar in some dbs
    "o": "STRING",
    "float32": "FLOAT",
    "float64": "STRING",
    "int32": "INTEGER",
    "int64": "INTEGER",
}

##  Explicit schema creation

we'll derive this from the tables for this exercise, but often, when you structure your DB, you know ahead of time what it is--the information could be stored in a CSV, configuration file, explicit in code, etc.

Here, we'll create the table, specify the primary key (which can be one or more columns), and append to the table.

More from [SQLite Tutorial](https://www.sqlitetutorial.net/sqlite-primary-key/), where you can see how to formulate a primary key from multiple attributes

In [265]:
df_to_write.dtypes.to_dict()

{'primary_id': dtype('int64'),
 'design_id': dtype('int64'),
 'strategy_id': dtype('int64'),
 'future_id': dtype('int64')}

In [266]:
# initialize the columns and specify the primary key
columns = []
primary_key = "primary_id"

for x in df_to_write.columns:
    dtype_convert = df_to_write.dtypes.to_dict().get("primary_id")
    sqlite_dtype = dict_dtypes_to_sql_types.get(str(dtype_convert))

    if sqlite_dtype is None:
        raise RuntimeError(f"Missing data type for column {x}")

    col = f"{x} {sqlite_dtype}"
    if x == primary_key:
        col = f"{col} NOT NULL PRIMARY KEY"
        
    columns.append(col)


# 
# make the schema
query_create_schema = ", ".join(columns)
query_create_schema = f"CREATE TABLE {table_name} ({query_create_schema});";

print(query_create_schema)

CREATE TABLE ATTRIBUTE_PRIMARY (primary_id INTEGER NOT NULL PRIMARY KEY, design_id INTEGER, strategy_id INTEGER, future_id INTEGER);


In [267]:
with engine.connect() as con:
    con.execute(sqlalchemy.text(query_create_schema))
    con.commit()

In [268]:
table_name in sqlalchemy.inspect(engine).get_table_names() 

True

In [269]:
# let's insert a row as a test
query_insert = ", ".join(list(df_to_write.columns))
query_insert = f"INSERT INTO {table_name} ({query_insert}) VALUES (0, 0, 0, 0);"
with engine.connect() as con:
    con.execute(sqlalchemy.text(query_insert))
    con.commit()

In [271]:
# check the transaction
with engine.connect() as con:
    df_check = pd.read_sql_query(f"SELECT * FROM {table_name}", con)
    
df_check

,primary_id,design_id,strategy_id,future_id
0,0,0,0,0


In [272]:
df_to_write

,primary_id,design_id,strategy_id,future_id
0,0,0,0,0
1,40,0,0,40
2,14014,0,1014,0
3,14054,0,1014,40
4,118118,0,5001,0
5,118158,0,5001,40
6,121121,0,5004,0
7,121161,0,5004,40
8,123123,0,5006,0
9,123163,0,5006,40


##  what happens when we try to commit the existing table?

In [273]:
# we can use the pd.to_sql function to write to the database
df_to_write.to_sql(
    table_name, 
    engine, 
    if_exists = "append", 
    index = None,
);


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: ATTRIBUTE_PRIMARY.primary_id
[SQL: INSERT INTO "ATTRIBUTE_PRIMARY" (primary_id, design_id, strategy_id, future_id) VALUES (?, ?, ?, ?)]
[parameters: [(0, 0, 0, 0), (40, 0, 0, 40), (14014, 0, 1014, 0), (14054, 0, 1014, 40), (118118, 0, 5001, 0), (118158, 0, 5001, 40), (121121, 0, 5004, 0), (121161, 0, 5004, 40)  ... displaying 10 of 12 total bound parameter sets ...  (125125, 0, 5008, 0), (125165, 0, 5008, 40)]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [276]:
# get existing values in the primary key
with engine.connect() as con:
    existing_keys = pd.read_sql_query(f"SELECT DISTINCT {primary_key} FROM {table_name};", con)

in_db = existing_keys[primary_key].to_numpy()

in_db

array([0])

In [277]:
# try removing the existing key
(
    df_to_write[
        ~df_to_write[primary_key].isin(in_db)
    ].to_sql(
        table_name, 
        engine, 
        if_exists = "append", 
        index = None,
    )
);


In [278]:
# check the transaction
with engine.connect() as con:
    df_check = pd.read_sql_query(f"SELECT * FROM {table_name}", con)
    
df_check

,primary_id,design_id,strategy_id,future_id
0,0,0,0,0
1,40,0,0,40
2,14014,0,1014,0
3,14054,0,1014,40
4,118118,0,5001,0
5,118158,0,5001,40
6,121121,0,5004,0
7,121161,0,5004,40
8,123123,0,5006,0
9,123163,0,5006,40


In [206]:
# finally, we can remove the table

In [279]:
with engine.connect() as con:
    con.execute(sqlalchemy.text(f"DROP TABLE {table_name};"))
    con.commit()

In [280]:
table_name in sqlalchemy.inspect(engine).get_table_names() 

False

###  Queries that modify the tables must be commited
- Think about the ACID principles
- Transactions can be executed, but don't necessarily take action until commited
- Can explicitly commit or use autommit option with sqlalchemy.text
- For more on committing, see [SQLAlchemy Autocommit](https://docs.sqlalchemy.org/en/20/core/connections.html#understanding-autocommit)

In [ ]:
if query is not None:
    with db_engine.connect() as con:
        con.execute(sqlalchemy.text(query))
        con.commit()

# Assignment

We are going to use SQLAlchemy to:

- Read some CSVs
- Create an SQLite database
- Create the tables associated with each CSV
- Write data in the CSVs to its associated table
- Execute some SQL queries to answer some questions

## 1. Check `path_tables` to ensure that all CSVs are unzipped

For the purposes of this exercise, one or more large CSVs are stored as a zip in `path_tables`. Use zipfile package to decompress any large CSVs.


In [ ]:
# here's the basic structure - https://stackoverflow.com/questions/3451111/unzipping-files-in-python
import zipfile

"""
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
     zip_ref.extractall(directory_to_extract_to)
"""

In [122]:
# TODO: unzip any zip files in path_tables

##  2. Read in the tables

- Set up code to read all CSV tables **without explicitly specifying their name** in `path_tables`
- Get a list of tables to read in from the directory
- Assign the tables to a dictionary

In [295]:
# TODO: get all CSV files in path_tables
#   - specify a regular expression using re.compile to match on
#      - use regex.match(string) to check if a string matches 
#      - returns None if there's no valid match

regex = re.compile("(.*\w).csv$")
regex.match("this is a name.csv3") is not None


False

In [39]:
# TODO: read tables in to an object that can be associated each table with its name

##  3. Create an SQLite database


In [17]:
# TODO: create the SQLite database 
#     NOTE: if you use sqlalchemy.create_engine("sqlite:///{path}") and the database doesn't exist, 
#     it will create the file)



### Create tables and schema

- Remember, we have to specify types; how do we translate types from pandas to those in 
- Need to use the `ALTER TABLE` query to assign primary keys to each table after it's complete. See [w3schools](https://www.w3schools.com/sql/sql_primarykey.ASP) for information on how to alter the table to add the primary key. **NOTE** a number of tables have multi-attribute primary keys. 

- Write a function(s) that can take a `pd.DataFrame` and generate this query


In [ ]:
# TODO: create a function to create a database schema based on columns in the CSV

In [ ]:
# TODO: create a function to write a table to the database using the data, table name, and engine

In [ ]:
# TODO: write each table to the database

##  4. Execute queries using SQL Alchemy

The data store model runs (inputs and outputs) for an emissons model in different countries in Latin America across different policy sets (indexed by `strategy_id`) and assumptions about potential future states of the world (`future_id`). 

In `MODEL_OUTUPTS`, you can use the sum of all columns that start with `emission_co2e_subsector_total` as the net emissions. You may need to add this sum to your table.

Once you've populated the database, use the SQL Queries we've talked about to answer the following questions:


1. What are the top five emitting countries in 2015? How about in 2050? What are the bottom five in 2015 and 2050?
2. What are the average baseline (strategy name is `**baseline NDP**`) emissions in 2030, 2040, and 2050 for countries whose population center is below the equator? How about above the equator? How about for the strategy called `**PFLO: all transformations with partial land use allocation**`?
    - **NOTES**
        - the `ATTRIBUTE_REGION` table contains this information for each region to identify population centers above and below the equator
        - the `ATTRIBUTE_STRATEGY` table contains information about the names
3. In 2050 and for the `**PFLO: all transformations with partial land use allocation**` strategy, which country sees the largest difference in emissions between future 0 and future 40?



In [298]:
df_in = pd.read_csv(path_tables.joinpath("ATTRIBUTE_REGION.csv"))

In [299]:
df_in

,category_name,region,iso_alpha_3,iso_alpha_2,iso_numeric,fao_area_code,weo_country_code,ipcc_ar5_region,ipcc_estimated_afolu_region,un_region,un_sub_region,world_bank_global_region,latitude_population_centroid_2020,longitude_population_centroid_2020
0,Afghanistan,afghanistan,AFG,AF,4,2,512,South Asia,South Asia (Indian subcontinent),Asia,Southern Asia,South Asia,34.612592,67.507387
1,Albania,albania,ALB,AL,8,3,914,Economies in Transition,Eastern Europe,Europe,Southern Europe,Europe & Central Asia,41.206902,19.858921
2,Algeria,algeria,DZA,DZ,12,4,612,Middle East and North Africa,Near East (Middle East) and North Africa,Africa,Northern Africa,Middle East & North Africa,35.524570,3.494062
3,American Samoa,american_samoa,ASM,AS,16,5,-999,South-East Asia and Pacific,Oceania,Oceania,Polynesia,East Asia & Pacific,-14.309417,-170.690438
4,Andorra,andorra,AND,AD,20,6,171,Western Europe,Western Europe,Europe,Southern Europe,Europe & Central Asia,42.524074,1.562281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Viet Nam,viet_nam,VNM,VN,704,237,582,South-East Asia and Pacific,East Asia and South-East Asia,Asia,South-eastern Asia,East Asia & Pacific,15.441861,106.418389
213,Yemen,yemen,YEM,YE,887,249,474,Middle East and North Africa,Near East (Middle East) and North Africa,Asia,Western Asia,Middle East & North Africa,14.735593,44.434270
214,Zambia,zambia,ZMB,ZM,894,251,754,Sub Saharan Africa,Sub-Saharan Africa,Africa,Sub-Saharan Africa,Sub-Saharan Africa,-13.593902,28.635402
215,Zimbabwe,zimbabwe,ZWE,ZW,716,181,698,Sub Saharan Africa,Sub-Saharan Africa,Africa,Sub-Saharan Africa,Sub-Saharan Africa,-18.652425,30.618165
